In [1]:
import numpy as np
from sage.all import *
from sage.modules.misc import gram_schmidt
from scipy.io import savemat

Construct the  standard basis vectors corresponding with $ \Phi$ according to the SYT $\theta$'s order. For example, take $k=5 $, $d=3$, $\Phi$ as 
    [[1,2,2],[3,3]],and $\theta$ as [[1,2,4],[3,5]], then $e_{\Phi}=e_1 \otimes e_2 \otimes e_3 \otimes e_2 \otimes e_3 $.

In [2]:
def get_coordinate_order(syt):
    positions = []
    for row_idx, row in enumerate(syt):
        for col_idx, num in enumerate(row):
            positions.append((num, (row_idx, col_idx)))
    positions.sort(key=lambda x: x[0])
    coords = [coord for num, coord in positions]
    return coords

def generate_basis(syt, ssyt,d):
    if len(syt) != len(ssyt):
        raise ValueError("SYT and SSYT have different number of rows.")
    for i in range(len(syt)):
        if len(syt[i]) != len(ssyt[i]):
            raise ValueError(f"Row {i} has different lengths in SYT and SSYT.")
    coords = get_coordinate_order(syt)
    indices = []
    for (row, col) in coords:
        indices.append(ssyt[row][col])
    basis = [i for i in indices]
    
    # Initialize an empty list to store the basis vectors
    basis_vectors = []

    # Traverse the matrix in row-major order
    #for row in SSYT:
    for element in basis:
            # Create the corresponding standard basis vector (unit vector)
            e_i = np.zeros(d)
            e_i[element - 1] = 1  # Set the correct index to 1 (1-based index)
            basis_vectors.append(e_i)  # Append the basis vector
            
    
    return basis_vectors

In [3]:
#usage example

SYT=[[1, 2,4], [3,5]]
SSYT=[[1, 2,2], [3,3]]
print('corresponding standard basis',generate_basis(SYT, SSYT,3))

corresponding standard basis [array([1., 0., 0.]), array([0., 1., 0.]), array([0., 0., 1.]), array([0., 1., 0.]), array([0., 0., 1.])]


Implement the action of a permutation group element on a basis:

In [4]:
# Implement the action of a permutation group element on a basis
def compute_tensor_product(permutationmatrix, SYT, SSYT, d):
    
    vec_list = generate_basis(SYT, SSYT, d)
    
    # Calculate the result of applying permutationmatrix to a vector v
    # For each row i, compute sum_{j} permutationmatrix[i, j]*vec_list[j]
    result_vec = []
    for i in range(permutationmatrix.shape[0]):
        temp = np.zeros(d)  # Initialize a d-dimensional zero vector [0, 0,...,0]
        for j in range(permutationmatrix.shape[1]):
            temp += permutationmatrix[i, j] * vec_list[j]
        result_vec.append(temp)
    
    
    # Take the tensor product (Kronecker product) of the resulting vectors
    tensor_prod = result_vec[0]
    for vec in result_vec[1:]:
        tensor_prod = np.kron(tensor_prod, vec)
    
    
    return tensor_prod


In [5]:
# Example usage:
p_matrix=np.array([[0, 0, 1],
        [0, 1, 0],
        [1, 0, 0]])
d=2
SYT=[[1, 3], [2]]
SSYT=[[1, 1], [2]]
compute_tensor_product(p_matrix,SYT,SSYT,d)

array([0., 0., 1., 0., 0., 0., 0., 0.])

$Y_{\theta_i}(e_{\Phi_j})$ is a non-orthonormalized Schur basis, where $Y_{\theta_i}$ is the Young symmetrizer related to SYT $\theta_i$, $e_{\Phi_j}$ is standard basis vectors corresponding with $ \Phi$ according to the SYT $\theta_i$'s order.

In [6]:
def non_orthonormalized_Schur_basis(SYT, SSYT, d, star=0):
    r"""
    Returns a non-orthonormalized Schur basis.
    """
    
    t = SkewTableau(SYT)
    if star:
        t = t.restrict(t.size() - star)
    n = t.size()

    # Get the original group algebra elements
    rs = t.row_stabilizer().list()
    cs = t.column_stabilizer().list()
    QSn = SymmetricGroupAlgebra(QQ, n)
    

    # Construct symmetric and antisymmetric elements
    sym = QSn.sum(QSn(Permutation(h)) for h in rs)
    antisym = QSn.sum(QQ(Permutation(v).sign()) * QSn(Permutation(v)) for v in cs)
   
    res = QSn.right_action_product(antisym, sym)
    

    # Handle the special case for empty tableaux
    if n <= 1:
        return MatrixSpace(QQ, d**n, d**n).identity_matrix()

    
    result = np.zeros((d**n,))
    for p, coeff in res.monomial_coefficients().items():
        
        result += coeff * compute_tensor_product(np.array(p.to_matrix()), SYT, SSYT, d)

    return result.reshape(-1, 1)


In [7]:
# Example usage:
SYT=[[1, 3], [2]]
SSYT=[[1, 1], [2]]
non_orthonormalized_Schur_basis(SYT,SSYT, 2)

array([[ 0.],
       [-1.],
       [ 2.],
       [ 0.],
       [-1.],
       [ 0.],
       [ 0.],
       [ 0.]])

$(\mathbb{C}^d)^{\otimes n}\cong\bigoplus_{\lambda\in Y_d^n}\mathbb{Q}_\lambda\otimes \mathbb{P}_\lambda$

In [8]:
def schur_transform(d, n):
    schur_transform_matrix = []
    V = VectorSpace(QQ, d**n)
    
    for lam in Partitions(n, max_length=d):
         
        st = StandardTableaux(lam)
        SYT_list = st.list()
        sst = SemistandardTableaux(lam, max_entry=d)
        SSYT_list = sst.list()
        for SYT in SYT_list:
            for SSYT in SSYT_list:
                schur_transform_matrix.append(non_orthonormalized_Schur_basis(SYT, SSYT, d).tolist())
                
    schur_basis = [V(vec) for vec in schur_transform_matrix]
    
    # Use the gram_schmidt function to orthogonalize;
    # orthonormal_schur_basis is the matrix of orthogonalized basis vectors
    # mu is the Gram-Schmidt coefficients
    orthonormal_schur_basis, mu = gram_schmidt(schur_basis)

    
    # Normalize
    normalized_vectors = [v / np.linalg.norm(v) for v in orthonormal_schur_basis]
            
    return normalized_vectors


In [9]:
# Example usage:
d=2
n=2

U_sch=np.array(schur_transform(d,n))
savemat("U_sch_d2n2.mat", {"U_sch_d2n2": U_sch.T})
print('schur_transform: \n',U_sch.T)

#with open("U_sch_d3n8.m", "w") as f:
    #f.write("U_sch_d3n8 = [\n")
    #for row in U_sch.T:
        #f.write("  " + " ".join(map(str, row)) + ";\n")
    #f.write("];\n")
# print('Verify if U_sch is a unitary matrix: \n',U_sch @ U_sch.conj().T )
# print('Verify if U_sch is a unitary matrix: \n', U_sch.conj().T @ U_sch )

schur_transform: 
 [[ 1.          0.          0.          0.        ]
 [ 0.          0.70710678  0.          0.70710678]
 [ 0.          0.70710678  0.         -0.70710678]
 [ 0.          0.          1.          0.        ]]
